> 分析各种方案最后生成的reportTotalDf.csv
>> 分析各媒体的R7 MAPE（按天，按月）   
>> 分析各媒体的增长倍率MAPE（按天，按月）   
>> 分析各媒体的首日、7日回收 方差   
>> 分析各媒体的首日、7日回收占比 方差   

`jupyter-lab --allow-root --ip 192.168.40.62`


In [ ]:
import pandas as pd
import numpy as np

import os
# import sys
# sys.path.append('/src')


In [ ]:
def getFilename(filename):
    return '/src/data/customLayer/%s.csv'%(filename)

In [ ]:
mediaList = [
    {'name':'bytedance','codeList':['bytedanceglobal_int'],'sname':'Bd'},
    {'name':'facebook','codeList':['Social_facebook','restricted','Facebook Ads','facebook','FacebookAds'],'sname':'Fb'},
    {'name':'google','codeList':['googleadwords_int'],'sname':'Gg'},
    {'name':'unknown','codeList':[],'sname':'Og'}
]

In [ ]:
iosMediaList = [
    {'name':'applovin','codeList':['applovin_int'],'sname':'Al'},
    {'name':'bytedance','codeList':['bytedanceglobal_int'],'sname':'Bd'},
    {'name':'facebook','codeList':['Social_facebook','restricted','Facebook Ads','facebook','FacebookAds'],'sname':'Fb'},
    {'name':'google','codeList':['googleadwords_int'],'sname':'Gg'},
    {'name':'unknown','codeList':[],'sname':'Og'}
]

In [ ]:
def mediaR7MapeByDay(mediaName,totalDf):
    r7RealIndex = '%s revenue7(real)'%(mediaName)
    r7RPredIndex = '%s revenue7(pred)'%(mediaName)
    mape = pd.DataFrame({'mape':(totalDf[r7RealIndex] - totalDf[r7RPredIndex])/totalDf[r7RealIndex]})
    mape.loc[mape.mape < 0,'mape'] *= -1
    return mape['mape'].mean()

In [ ]:
def mediaR7MapeByMonth(mediaName,totalDf):
    r7RealIndex = '%s revenue7(real)'%(mediaName)
    r7RPredIndex = '%s revenue7(pred)'%(mediaName)

    totalDf.loc[:,'install_month'] = totalDf['install_date_group'].str.slice(0,7)
    totalMpnthDf = totalDf.groupby(by=['install_month'],as_index=False).agg('sum')

    # print(totalMpnthDf)

    mape = pd.DataFrame({'mape':(totalMpnthDf[r7RealIndex] - totalMpnthDf[r7RPredIndex])/totalMpnthDf[r7RealIndex]})
    mape.loc[mape.mape < 0,'mape'] *= -1
    return mape['mape'].mean()

In [ ]:
# 增长倍率 r7/r1
def mediaP7MapeByDay(mediaName,totalDf):
    r7RealIndex = '%s r7/r1(real)'%(mediaName)
    r7RPredIndex = '%s r7/r1(pred)'%(mediaName)
    mape = pd.DataFrame({'mape':(totalDf[r7RealIndex] - totalDf[r7RPredIndex])/totalDf[r7RealIndex]})
    mape.loc[mape.mape < 0,'mape'] *= -1
    return mape['mape'].mean()

In [ ]:
def mediaP7MapeByMonth(mediaName,totalDf):
    r7RealIndex = '%s r7/r1(real)'%(mediaName)
    r7RPredIndex = '%s r7/r1(pred)'%(mediaName)

    totalDf.loc[:,'install_month'] = totalDf['install_date_group'].str.slice(0,7)
    totalMpnthDf = totalDf.groupby(by=['install_month'],as_index=False).agg('sum')

    # print(totalMpnthDf)

    mape = pd.DataFrame({'mape':(totalMpnthDf[r7RealIndex] - totalMpnthDf[r7RPredIndex])/totalMpnthDf[r7RealIndex]})
    mape.loc[mape.mape < 0,'mape'] *= -1
    return mape['mape'].mean()

In [ ]:
# 媒体 首日 回收 的方差
def mediaR1Var(mediaName,totalDf):
    index = '%s revenue1(real)'%(mediaName)

    return totalDf[index].var()

In [ ]:
# 媒体 7日 回收 的方差
def mediaR7Var(mediaName,totalDf):
    index = '%s revenue7(real)'%(mediaName)

    return totalDf[index].var()

In [ ]:
# 媒体 首日 回收占比 的方差
def mediaP1Var(mediaName,totalDf):
    index = '%s revenue1(real)'%(mediaName)
    p1 = totalDf[index]/totalDf['revenue1']
    return p1.var()

In [ ]:
# 媒体 7日 回收占比 的方差
def mediaP7Var(mediaName,totalDf):
    index = '%s revenue7(real)'%(mediaName)
    p1 = totalDf[index]/totalDf['revenue7']
    return p1.var()

In [ ]:
# 由于要出iOS和android两种报告，媒体信息略有不同，需要将媒体列表当做参数
def report(docDirname,mediaList):
    totalDf = pd.read_csv(os.path.join(docDirname,'reportTotalDf.csv'))
    # 计算首日回收总额
    print(docDirname)
    print('MAPE:%2f%%'%(totalDf['mape'].mean()))
        
    isIOS = False
    for media in mediaList:
        mediaName = media['name']
        if '%s revenue1(real)'%(mediaName) not in totalDf.columns:
            isIOS = True
            break
    
    if isIOS == True:
        # iOS 版本是没有首日付费金额的，只能用首日CV推算
        for media in iosMediaList:
            mediaName = media['name']
            mediaDf = pd.read_csv(getFilename('step3_mediaSumDf_%s'%mediaName))
            totalDf.loc[:,'%s revenue1(real)'%(mediaName)] = mediaDf['usd']

    totalDf.loc[:,'revenue1'] = 0
    for media in mediaList:
        mediaName = media['name']
        if '%s revenue1(real)'%(mediaName) not in totalDf.columns:
            # iOS 版本是没有首日付费金额的，只能用首日CV推算
            for media in iosMediaList:
                mediaName = media['name']
                mediaDf = pd.read_csv(getFilename('step3_mediaSumDf_%s'%mediaName))
                totalDf.loc[:,'%s revenue1(real)'%(mediaName)] = mediaDf['usd']

        totalDf.loc[:,'revenue1'] += totalDf['%s revenue1(real)'%(mediaName)]
    # 分析各媒体的R7 MAPE 按天
    for media in mediaList:
        mediaName = media['name']
        print(mediaName,':')
        if isIOS == False:
            print('%s r7 by day MAPE: %.2f%%'%('',mediaR7MapeByDay(mediaName,totalDf)*100))
            print('%s r7 by month MAPE: %.2f%%'%('',mediaR7MapeByMonth(mediaName,totalDf)*100))
            print('%s r7/r1 by day MAPE: %.2f%%'%('',mediaP7MapeByDay(mediaName,totalDf)*100))
            print('%s r7/r1 by month MAPE: %.2f%%'%('',mediaP7MapeByMonth(mediaName,totalDf)*100))

        print('%s r1 var by day: %.2f'%('',mediaR1Var(mediaName,totalDf)))
        if isIOS == False:
            print('%s r7 var by day: %.2f'%('',mediaR7Var(mediaName,totalDf)))

        print('%s p1 var by day: %.4f'%('',mediaP1Var(mediaName,totalDf)))
        if isIOS == False:
            print('%s p7 var by day: %.4f'%('',mediaP7Var(mediaName,totalDf)))

        
        


In [ ]:
# V1
report('/src/data/doc/customLayer//iOSCustom_20230316_082543',mediaList)

In [ ]:
# V1.1
report('/src/data/doc/customLayer//iOSCustom_20230316_060009',mediaList)

In [ ]:
# iOS V2
report('/src/data/doc/customLayer//iOSCustom_20230316_103636',iosMediaList)